In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction to Vertex AI Vector Search 2.0

This notebook provides a comprehensive introduction to **[Vertex AI Vector Search 2.0](https://cloud.google.com/vertex-ai/docs/vector-search-2/overview)** for developers who are familiar with vector search and embeddings concepts, but new to this Google Cloud service.

**New to vector search and embeddings?** If you're looking to learn the basics, please refer to: [Introduction to Text Embeddings and Vector Search](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro-textemb-vectorsearch.ipynb)

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/embeddings/vector-search-2-intro.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fembeddings%2Fvector-search-2-intro.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/vector-search-2-intro.ipynb">
      <img width="32px" src="https://raw.githubusercontent.com/primer/octicons/refs/heads/main/icons/mark-github-24.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## What is Vector Search 2.0?

[Vector Search 2.0](https://cloud.google.com/vertex-ai/docs/vector-search-2/overview) is Google Cloud's fully managed, self-tuning vector database built on Google's [ScaNN (Scalable Nearest Neighbors)](https://github.com/google-research/google-research/tree/master/scann) algorithm - the same technology powering Google Search, YouTube, and Google Play.

### Key Differentiators

- **Zero Indexing to Billion-Scale Index**: Start developing immediately with zero indexing time using [kNN (k-Nearest Neighbors)](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm), then scale to billions of vectors with millisec latency with Google-scale [ANN (Approximate Nearest Neighbor)](https://en.wikipedia.org/wiki/Nearest_neighbor_search#Approximate_nearest_neighbor) indexes for production - all with the same API and same dataset
- **Unified Data Storage**: Store both [vector embeddings](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings) and user provided data together (no separate database or feature store needed)
- **Auto-Embeddings**: Automatically generate semantic embeddings using [Vertex AI embedding models](https://docs.cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings#google-models)
- **Built-in Full Text Search**: Provides a built-in [full-text search](https://cloud.google.com/discover/what-is-full-text-search?e=48754805&hl=en) without needing to generate sparse embeddings by yourself. You can also choose to use your own sparse embeddings (e.g., BM25, SPLADE) with Vector Search for a customized full-text search.
- **Hybrid Search**: Combine semantic and keyword/token-based search in a single query with intelligent [RRF](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) ranking
- **Self-Tuning**: Auto-optimized performance without manual configuration
- **Enterprise-Ready**: Built-in scalability, security, and compliance

### Core Architecture

Vector Search 2.0 has three main components:

1. **[Collections](https://cloud.google.com/vertex-ai/docs/vector-search-2/collections/collections)**: Schema-enforced containers for your data
2. **[Data Objects](https://cloud.google.com/vertex-ai/docs/vector-search-2/data-objects/data-objects)**: Individual items with data and vector embeddings
3. **[Indexes](https://cloud.google.com/vertex-ai/docs/vector-search-2/indexes/indexes)**: Instant nearest neighbor search your data with kNN. For low latency nearest neighbor search use an ANN index.
   - **Start fast**: Use kNN immediately with zero setup time - perfect for development and small datasets
   - **Scale to production**: Use ANN indexes for billion-scale search with sub-second latency powered by ScaNN algorithm

Let's explore each concept with hands-on examples!


## Example Scenario: E-Commerce Product Search

To demonstrate Vector Search 2.0's capabilities, we'll build a **product search and recommendation system** using the TheLook e-commerce dataset.

**For this demo**: We'll use a **3,000 product sample** for faster processing and lower costs. The techniques shown here scale seamlessly to the full dataset (~30K products) or even larger catalogs.

### Business Use Cases:

1. **Product Discovery**: Find similar products based on product name semantics
2. **Semantic Search**: "Find products similar to 'blue denim jeans'"
3. **Filtered Shopping**: "Show me Dresses under $100"
4. **Hybrid Search**: Combine semantic similarity with keyword matching for better product recommendations

### Dataset Overview:

The full TheLook dataset contains **29,120 fashion products** from an e-commerce platform with the following attributes:

- `id`: Product ID (e.g., "8037")
- `name`: Product name (e.g., "Jostar Short Sleeve Solid Stretchy Capri Pants Set")
- `category`: Product category (26 categories: Dresses, Jeans, Tops & Tees, etc.)
- `retail_price`: Product price in USD (e.g., 38.99)

### Sample Data:

| ID | Name | Category | Price |
|-----|------|----------|-------|
| 8037 | Jostar Short Sleeve Solid Stretchy Capri Pants Set | Clothing Sets | $38.99 |
| 8036 | Womens Top Stitch Jacket and Pant Set by City Lights | Clothing Sets | $199.95 |
| 8035 | Ulla Popken Plus Size 3-Piece Duster and Pants Set | Clothing Sets | $159.00 |

Throughout this notebook, we'll walk through each step from setting up Collections and adding products, to performing various types of searches, and finally optimizing with indexes. Let's get started!

-----

## Prerequisites

This tutorial requires a Google Cloud project that is linked with a billing account. To create a new project, take a look at [this document](https://cloud.google.com/vertex-ai/docs/start/cloud-environment) to create a project and setup a billing account for it.
To get the permissions that you need to give a service account access to enable APIs and interact with Vertex AI resources, ask your administrator to grant you the [Security Admin](https://cloud.google.com/iam/docs/roles-permissions/iam#iam.securityAdmin) (`roles/iam.securityAdmin`) IAM role on your project. For more information about granting roles, see [Manage access to projects, folders, and organizations](https://cloud.google.com/iam/docs/granting-changing-revoking-access).

### Important: Resource Cleanup

Vector Search 2.0 resources incur costs when active. Make sure to run the cleanup section at the end of this tutorial to delete all Collections and Indexes.

## Install the Vector Search SDK

First, we'll install the `google-cloud-vectorsearch` Python package.


In [ ]:
%pip install google-cloud-vectorsearch tqdm

Then, set `PROJECT_ID` for your Google Cloud project. You can leave the `LOCATION` as the default value.

In [ ]:
# Set PROJECT_ID and LOCATION
PROJECT_ID = "your-project-id"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Validate PROJECT_ID is set
if PROJECT_ID == "your-project-id" or not PROJECT_ID or PROJECT_ID == "":
    raise ValueError(
        "⚠️ Please set PROJECT_ID to your actual Google Cloud project ID in the cell above"
    )

print(f"✅ Using project: {PROJECT_ID}")
print(f"✅ Using location: {LOCATION}")

## Authentication

On Colab, run the following to authenticate calls to the Vector Search APIs. For Colab Enterprise and Cloud Workbench, you can skip this part.

In [8]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

## Enable APIs

Run the following commands to enable APIs for Vector Search and, if using Auto-Embeddings or Semantic Search, the Vertex AI API with this Google Cloud project.


In [ ]:
! gcloud services enable vectorsearch.googleapis.com aiplatform.googleapis.com --project "{PROJECT_ID}"

-----

# Part 1: Collections - Your Data Container

In this part, you'll learn how to create and configure **Collections** - the foundation of Vector Search 2.0. Collections are schema-enforced containers that define the structure of your data and embeddings.

**What you'll accomplish:**
- Initialize the Vector Search SDK clients
- Understand Collection schemas (data schema + vector schema)
- Create a product Collection with auto-embeddings
- Inspect and verify your Collection configuration

Let's start by setting up the SDK clients!

## SDK Clients Overview

The Vector Search 2.0 SDK uses a **modular client architecture** to organize operations by function. Instead of one monolithic client, you'll work with three specialized service clients throughout this tutorial:

1. **VectorSearchServiceClient**: Manages Collections and Indexes (CRUD operations)
2. **DataObjectServiceClient**: Manages Data Objects (create, update, delete)
3. **DataObjectSearchServiceClient**: Performs search and query operations

This separation provides clear boundaries between data management and search operations. For more details, see the [Python SDK Documentation](https://cloud.google.com/python/docs/reference/vectorsearch/latest).

To begin with the product, let's create client objects for them.

In [ ]:
from google.cloud import vectorsearch_v1beta

vector_search_service_client = vectorsearch_v1beta.VectorSearchServiceClient()
data_object_service_client = vectorsearch_v1beta.DataObjectServiceClient()
data_object_search_service_client = vectorsearch_v1beta.DataObjectSearchServiceClient()


## What is a Collection?

A **[Collection](https://cloud.google.com/vertex-ai/docs/vector-search-2/collections/collections)** is a schema-enforced container for your data in Vector Search 2.0. Think of it as a table in a traditional database, but optimized for vector operations.

### Key Concepts:

- **[Data Schema](https://cloud.google.com/vertex-ai/docs/vector-search-2/collections/collections#data-schema)**: Defines the structure of your data ([JSON Schema](https://json-schema.org/) format). Note: We do not currently support `additionalProperties=True`.
- **[Vector Schema](https://cloud.google.com/vertex-ai/docs/vector-search-2/collections/collections#vector-schema)**: Defines your embedding fields with their dimensions and types
- **Schema Enforcement**: All Data Objects must conform to the defined schemas
- **Multiple Embeddings**: You can have multiple vector fields per object (e.g., text_embedding, image_embedding)

### Collection Features:

1. **Dense Vectors**: Standard continuous embeddings (e.g., [0.1, 0.2, 0.3, ...])
3. **[Auto-Embeddings](https://cloud.google.com/vertex-ai/docs/vector-search-2/data-objects/data-objects#auto-populate-embeddings)**: Automatic embedding generation using Vertex AI models
4. **Flexible Data**: Store any JSON-compatible data alongside vectors

Now let's create our product Collection with all the schemas we need!

## Creating the Product Collection

Let's create our first Collection for the e-commerce product catalog. We'll define schemas that match our TheLook dataset structure and configure auto-embeddings for product names.


In [ ]:

import getpass
from datetime import datetime

collection_id = f"products-demo-{getpass.getuser()}-{datetime.now().strftime('%m%d%y-%H%M%S')}"
print(f"Collection ID: {collection_id}")

In [ ]:
# Create the product Collection with schemas that match our dataset

request = vectorsearch_v1beta.CreateCollectionRequest(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}",
    collection_id=collection_id,
    collection={
        # Data Schema: Product data (id, name, category, retail_price)
        "data_schema": {
            "type": "object",
            "properties": {
                "id": {"type": "string"},           # Product ID
                "name": {"type": "string"},         # Product name
                "category": {"type": "string"},     # Product category (Dresses, Jeans, etc.)
                "retail_price": {"type": "number"}, # Product price in USD
            },
        },
        # Vector Schema: Product name-based embeddings for semantic and keyword search
        "vector_schema": {
            # Dense embedding: Captures semantic meaning of product names
            # Auto-generated by Vertex AI using text-embedding-004 model
            "name_dense_embedding": {
                "dense_vector": {
                    "dimensions": 768,  # Standard dimension for text-embedding-004
                    "vertex_embedding_config": {
                        # Auto-generate dense embeddings from product name
                        "model_id": "text-embedding-004",
                        "text_template": "{name}",
                        "task_type": "RETRIEVAL_DOCUMENT",
                    },
                },
            },
        },
    }
)

operation = vector_search_service_client.create_collection(request=request)
operation.result()

### Key Points:

What we just accomplished:

✅ **Created a Collection** named `products-demo-{user}-{date}` with strict schemas  
✅ **Defined data schema** for 4 data fields: id, name, category, retail_price  
✅ **Configured dense vector embedding for product name-based semantic search**:
   - `name_dense_embedding` - **Auto-generated** by Vertex AI from product name using text-embedding-004 model

### Auto-Embeddings Feature

One powerful feature of Vector Search 2.0 is **automatic embedding generation**. When you create a Data Object without providing vectors for fields configured with `vertex_embedding_config` as we've done with `name_dense_embedding`, the service automatically generates them using Vertex AI models. This requires the Vertex AI API to be enabled (done in the setup section).

## Inspecting Collections

You can retrieve and list Collections to verify their configuration:

Let's check the Collection we just created to confirm all our schemas are in place.

In [ ]:
request = vectorsearch_v1beta.GetCollectionRequest(
    name=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}"
)
vector_search_service_client.get_collection(request)

-----
# Part 2: Data Objects - Your Actual Data

Now that we have a Collection set up, it's time to populate it with actual data! In this part, you'll learn how to add **Data Objects** - the individual items stored in your Collection.

**What you'll accomplish:**
- Download and prepare the TheLook e-commerce dataset (3,000 products)
- Understand the Data Object structure (id, data, vectors)
- Create individual Data Objects with auto-generated embeddings
- Perform efficient batch imports with rate limiting
- Learn best practices for managing embedding API quotas

By the end of this section, you'll have a fully populated Collection with 3,000 products, each with automatically generated semantic embeddings!

## Downloading theLook dataset

Now that our Collection is ready, we need data! Let's download the TheLook e-commerce dataset.

For this demo, we'll use a **randomly sampled 3,000 products** (from the full ~30K dataset) for faster demo. Random sampling ensures better category distribution compared to sequential selection. You can easily switch to the full dataset by changing `MAX_PRODUCTS = None` in the code below.

In [ ]:
import json
import urllib.request
import random

# Download and load TheLook e-commerce dataset with error handling
print("📥 Downloading TheLook e-commerce dataset...")
dataset_url = "https://storage.googleapis.com/gcp-samples-ic0-vs20demo/thelook_dataset.jsonl"

# For faster demo: randomly sample 3,000 products (full dataset has ~30K)
# To use the full dataset, set MAX_PRODUCTS = None
MAX_PRODUCTS = 3000

all_products = []
required_fields = ['id', 'name', 'category', 'retail_price']

# Load all products first
with urllib.request.urlopen(dataset_url) as response:
    for i, line in enumerate(response, 1):
        product_data = json.loads(line.decode('utf-8'))
        
        # Check if all required fields are present
        if all(field in product_data for field in required_fields):
            all_products.append({
                "id": product_data["id"],
                "data": {
                    "id": product_data["id"],
                    "name": product_data["name"],
                    "category": product_data["category"],
                    "retail_price": product_data["retail_price"],
                }
            })

# Random sampling for better category distribution
if MAX_PRODUCTS and len(all_products) > MAX_PRODUCTS:
    random.seed(42)  # Set seed for reproducibility
    products = random.sample(all_products, MAX_PRODUCTS)
    print(f"✅ Loaded and randomly sampled {len(products):,} products from {len(all_products):,} total")
    print(f"   (Random sampling ensures better category distribution)")
else:
    products = all_products
    print(f"✅ Loaded {len(products):,} products from TheLook dataset")

In [ ]:
# Inspect the first five product's structure
products[:5]

We will load this `products` data to the Collection with Data Objects.

## What is a Data Object?

A **Data Object** represents a single item in your Collection. Each Data Object consists of:

1. **data_object_id**: Unique identifier
2. **data**: Data fields (as defined in data_schema)
3. **vectors**: Embedding vectors (as defined in vector_schema)

In our e-commerce scenario, each Data Object represents one product with its data (id, name, category, retail_price) and product name-based vector embedding (dense semantic embedding, auto-generated).


### Creating Data Objects

You can add Data Objects in three ways:
1. **Single Create**: Add one product at a time (useful for new inventory) - **covered in this section**
2. **Batch Create**: Add multiple products efficiently (bulk catalog import) - **covered later in this section**
3. **GCS Import**: Bulk import from Google Cloud Storage (large-scale datasets) - **see [Vector Search 2.0 Quickstart](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/vector-search-2-quickstart.ipynb)**

In this tutorial, we'll focus on single object creation to understand the fundamentals. For production use cases with larger datasets, refer to the batch import method covered later in this section, or the GCS import method in the [Vector Search 2.0 Quickstart notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/vector-search-2-quickstart.ipynb).


## Create Single Data Object

Let's start by creating a single Data Object to understand the basic structure. This is useful for real-time updates or adding individual items one at a time.

In [ ]:
# Add the first product as a demonstration

request = vectorsearch_v1beta.CreateDataObjectRequest(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}",
    data_object_id=products[0]["id"],
    data_object={
        "data": products[0]["data"],  # Data: id, name, category, retail_price
        "vectors": {},  # Empty vectors - dense embedding will be auto-generated!
    },
)
result = data_object_service_client.create_data_object(request=request)
print(f"✅ Created Data Object: {products[0]['data']['name']}")
print(f"   Category: {products[0]['data']['category']} | Price: ${products[0]['data']['retail_price']:.2f}")
print(f"   💡 Dense embedding auto-generated by Vector Search 2.0 from product name")


### Get Data Object

Retrieve a specific Data Object by its ID for checking:

In [ ]:
request = vectorsearch_v1beta.GetDataObjectRequest(
        name=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}/dataObjects/{products[0]["id"]}",
    )
data_object_service_client.get_data_object(request=request)

You can see that the `name_dense_embedding` field is filled with embedding automatically.

### Delete Data Object

Make sure to delete this Data Object to avoid duplication with the data loaded with the following sample.

In [ ]:
delete_request = vectorsearch_v1beta.DeleteDataObjectRequest(
    name=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}/dataObjects/{products[0]["id"]}"
)
data_object_service_client.delete_data_object(delete_request)

## Batch Create Data Objects

Now that you understand how single Data Objects work, let's scale up to import our **entire 3,000 product catalog** efficiently using batch operations.

**Why Batch Operations?**
- **10-100x more efficient** than creating items one at a time
- Reduces API calls from 3,000 individual requests to just 30 batch requests
- Handles rate limiting automatically to stay within embedding API quotas
- Production-ready approach for importing large datasets

The code below demonstrates best practices for bulk data import: grouping products into batches of 100, implementing rate limiting, handling errors gracefully, and tracking progress with a progress bar.

In [ ]:
# Import ALL products from the dataset in batches of 100
# This efficiently handles the full ~3K product catalog

import time
from tqdm.auto import tqdm

batch_size = 100
delay_per_batch = 2.5  # Delay between batches to stay under API quota

# Process products in batches
for batch_start in tqdm(range(0, len(products), batch_size), desc="Importing products", unit="batch"):
    batch_end = min(batch_start + batch_size, len(products))
    
    # Prepare batch request
    batch_request = [
        {
            "data_object_id": product["id"],
            "data_object": {"data": product["data"], "vectors": {}},
        }
        for product in products[batch_start:batch_end]
    ]
    
    # Execute batch create
    try:
        request = vectorsearch_v1beta.BatchCreateDataObjectsRequest(
            parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}",
            requests=batch_request,
        )
        data_object_service_client.batch_create_data_objects(request)
    except Exception as e:
        if "already exists" not in str(e).lower():
            tqdm.write(f"⚠️ Batch {batch_start//batch_size + 1} error: {str(e)[:80]}")
    
    # Rate limiting
    if batch_end < len(products):
        time.sleep(delay_per_batch)

print(f"✅ Import complete!")

### Key Points

What this code does:

1. **Batching**: Groups 100 products per request for efficiency (~30x fewer API calls than individual creates)

2. **Rate Limiting**: Waits 2.5 seconds between batches to stay under the embedding API quota (3,000 requests/minute)

3. **Auto-Embeddings**: Empty `vectors: {}` triggers automatic embedding generation using the `text-embedding-004` model

4. **Duplicate Handling**: Silently skips "already exists" errors, allowing safe re-runs

5. **Progress Tracking**: Shows real-time progress with tqdm progress bar

### Auto-Embeddings and Rate Limiting

**Why Rate Limiting is Necessary:**

When you create Data Objects with empty `vectors: {}`, Vector Search 2.0 automatically calls the Vertex AI Text Embeddings API to generate embeddings. For example, the `text-embeddings-004` model for the paid tier 1 project has a default quota limit of **3,000 requests per minute (RPM)**. You can check the quota limit on the [Quotas and system limits](https://docs.cloud.google.com/docs/quotas/view-manage) page on the Console.

**What Happens Without Rate Limiting:**

Without the `time.sleep(2.5)` delay, the batch import would attempt to create all 30 batches (3,000 products ÷ 100 per batch) in rapid succession, resulting in **3,000 embedding API calls in just a few seconds** - far exceeding the quota.

You would see errors like:
```
429 Resource has been exhausted (e.g. check quota).
```

**Our Rate Limiting Strategy:**

- **Batch size**: 100 products = 100 embedding API calls per batch
- **Delay**: 2.5 seconds between batches
- **Effective rate**: 100 requests ÷ 2.5 seconds = 40 requests/second = **2,400 RPM** (80% of quota)

This keeps us safely under the 3,000 RPM limit while still processing products efficiently. For the full 3,000 product dataset, the import takes approximately **75 seconds** (30 batches × 2.5 seconds).

If the current quota is not enough for your dataset, you can request [a Quota adjustment](https://docs.cloud.google.com/docs/quotas/view-manage#requesting_higher_quota). After increasing the limit, you can adjust the rate limiting delay based on the new limit.

-----

# Part 3: Querying and Filtering Data

Now that we have our product catalog populated, let's learn how to retrieve data! This section covers filtering and querying based on data.

## Query vs Search

Vector Search 2.0 distinguishes between two operations:

- **Query**: Filter and retrieve Data Objects based on data (like SQL WHERE clause)
- **Search**: Find similar items based on vector similarity (ANN search)

In our e-commerce scenario:
- **Query** is used for: "Show me all Jeans under $100" or "Find products in the Dresses category"
- **Search** is used for: "Find products with similar names" or "Recommend products like this one"

Let's start with querying to understand our product catalog.

## Filtering with Query Operators

Vector Search 2.0 supports a [rich query language](https://cloud.google.com/vertex-ai/docs/vector-search-2/query-search/query#filter-syntax) for filtering:

**Comparison operators**: `$eq`, `$ne`, `$gt`, `$gte`, `$lt`, `$lte`  
**Logical operators**: `$and`, `$or`  
**Array operators**: `$in`, `$nin`, `$all`

Let's see some examples:

In [ ]:
# Example 1: Browse by category - Find all products in Jeans category
jeans_request = vectorsearch_v1beta.QueryDataObjectsRequest(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}",
    filter={"category": {"$eq": "Jeans"}},
    output_fields=vectorsearch_v1beta.OutputFields(data_fields=["*"]),
)
jeans = data_object_search_service_client.query_data_objects(jeans_request)
print("All Jeans products:")
print([p.data["name"][:50] + "..." for p in jeans][:5])  # Show first 5

In [ ]:
# Example 2: Price-based filtering - Affordable Jeans (under $75)
# Useful for: "What are affordable jeans in our catalog?"
affordable_jeans_request = vectorsearch_v1beta.QueryDataObjectsRequest(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}",
    filter={"$and": [{"category": {"$eq": "Jeans"}}, {"retail_price": {"$lt": 75}}]},
    output_fields=vectorsearch_v1beta.OutputFields(data_fields=["*"]),
)
affordable_jeans = data_object_search_service_client.query_data_objects(
    affordable_jeans_request
)
print("Jeans under $75:")
print([f"{p.data['name'][:40]}... (${p.data['retail_price']:.2f})" for p in affordable_jeans][:5])

In [ ]:
# Example 3: Category browsing with price exclusion
# Useful for: "Show me Dresses or premium Clothing Sets (over $150)"
nested_conditionals_request = vectorsearch_v1beta.QueryDataObjectsRequest(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}",
    filter={
        "$or": [
            {"category": {"$eq": "Dresses"}},
            {
                "$and": [
                    {"category": {"$eq": "Clothing Sets"}},
                    {"retail_price": {"$gte": 150}},
                ]
            },
        ]
    },
    output_fields=vectorsearch_v1beta.OutputFields(data_fields=["*"]),
)
nested_conditionals = data_object_search_service_client.query_data_objects(
    nested_conditionals_request
)
print("Dresses OR (Clothing Sets >= $150):")
print([f"{p.data['name'][:40]}... | {p.data['category']} | ${p.data['retail_price']:.2f}" for p in nested_conditionals][:5])

### Query with Aggregates

Beyond filtering individual products, you can also get aggregate statistics about your Collection - like counting total products, or analyzing distributions by category.

In [ ]:
aggregate_request = vectorsearch_v1beta.AggregateDataObjectsRequest(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}",
    aggregate="COUNT",
)
data_object_search_service_client.aggregate_data_objects(aggregate_request)

-----

# Part 4: Vector Search

This is where Vector Search 2.0 truly shines! We'll now move beyond data filtering to semantic similarity search using vector embeddings.

## Using kNN (k-Nearest Neighbors) Search

In this section, we'll use the **[kNN (k-Nearest Neighbors)](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)** algorithm for vector search. The best part of kNN is that **you don't need to build any index** - you can search on the Collection as soon as you import the data, with **zero indexing time**!

**kNN Advantages:**
- ✅ **Instant search**: No waiting for index creation
- ✅ **Perfect for development**: Test and iterate quickly
- ✅ **Ideal for small datasets**: Works great with up to tens of thousands of rows

**kNN Limitations:**
- ⚠️ **Latency increases with data size**: For datasets with over tens of thousands of rows, you'll see longer latency

**Production Recommendation:**
For production deployments with large-scale data, we **strongly recommend using ANN (Approximate Nearest Neighbor)** indexes, which provide blazingly fast vector search even with billions of rows. We'll cover ANN indexes in **Part 5**.

## Types of Search in Vector Search 2.0

Vector Search 2.0 supports multiple search modalities:

1. **[Semantic Search](https://docs.cloud.google.com/vertex-ai/docs/vector-search-2/query-search/search#semantic-search)**: Natural language queries (with auto-generated embeddings)
2. **[Text Search](https://docs.cloud.google.com/vertex-ai/docs/vector-search-2/query-search/search#text-search)**: Traditional keyword search
3. **[Hybrid Search](https://docs.cloud.google.com/vertex-ai/docs/vector-search-2/query-search/search#hybrid-search)**: Combine multiple search types with ranking
4. **[Vector Search](https://docs.cloud.google.com/vertex-ai/docs/vector-search-2/query-search/search#vector-search)**: Provide your own query vector for similarity search

Let's explore each type with our product data!

## 1. Semantic Search

With Semantic Search, you can use **natural language queries**. Vector Search 2.0 automatically converts your text to an embedding and runs a vector search.

**E-Commerce Scenario**: A user types "blue denim jeans" - the system finds products with semantically similar names.

In [ ]:
query_text = "blue denim jeans"

# Semantic search automatically generates embeddings from the query text
semantic_search_request = vectorsearch_v1beta.SearchDataObjectsRequest(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}",
    semantic_search=vectorsearch_v1beta.SemanticSearch(
        search_text=query_text,
        search_field="name_dense_embedding",  # The vector field to search
        task_type="RETRIEVAL_QUERY",
        top_k=5,
        output_fields=vectorsearch_v1beta.OutputFields(data_fields=["name", "category", "retail_price"]),
    ),
)

results = data_object_search_service_client.search_data_objects(semantic_search_request)

print(f"Semantic search results for '{query_text}':")
for i, result in enumerate(results, 1):
    print(f"{i}. {result.data_object.data['name'][:60]}...")
    print(f"   {result.data_object.data['category']} - ${result.data_object.data['retail_price']:.2f}")

## 2. Text Search (Keyword Matching)

Text Search provides traditional full-text search.

**Note**: For customized full-text search, you can also use your own sparse embeddings (e.g., BM25, SPLADE) with Vector Search.

**E-Commerce Scenario**: "Find products with 'denim' or 'jeans' in the name"

In [ ]:
query_text = "denim OR jeans"

text_search_request = vectorsearch_v1beta.SearchDataObjectsRequest(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}",
    text_search=vectorsearch_v1beta.TextSearch(
        search_text=query_text,  # Boolean operators supported
        data_field_names=["name"],  # Search in product name field
        top_k=5,
        output_fields=vectorsearch_v1beta.OutputFields(data_fields=["name", "category", "retail_price"]),
    ),
)
results = data_object_search_service_client.search_data_objects(text_search_request)
print("Text search results for 'denim OR jeans':")
for i, result in enumerate(results, 1):
    print(f"{i}. {result.data_object.data['name'][:60]}... - ${result.data_object.data['retail_price']:.2f}")

## 3. Hybrid Search - Combining Multiple Search Strategies

One of the most powerful features of Vector Search 2.0 is **Hybrid Search** - combining multiple search strategies using [Reciprocal Rank Fusion (RRF)](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) to produce better, more balanced results.

### Semantic Search + Text Search Example

In [ ]:
# Hybrid search: combine semantic and text searches with built-in RRF
query_text = "blue denim jeans"  # Same query for both searches

batch_search_request = vectorsearch_v1beta.BatchSearchDataObjectsRequest(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}",
    searches=[
        vectorsearch_v1beta.Search(
            semantic_search=vectorsearch_v1beta.SemanticSearch(
                search_text=query_text,
                search_field="name_dense_embedding",
                task_type="RETRIEVAL_QUERY",
                top_k=20,
                output_fields=vectorsearch_v1beta.OutputFields(data_fields=["id", "name", "category", "retail_price"]),
            )
        ),
        vectorsearch_v1beta.Search(
            text_search=vectorsearch_v1beta.TextSearch(
                search_text=query_text,
                data_field_names=["name"],
                top_k=20,
                output_fields=vectorsearch_v1beta.OutputFields(data_fields=["id", "name", "category", "retail_price"]),
            )
        ),
    ],
    combine=vectorsearch_v1beta.BatchSearchDataObjectsRequest.CombineResultsOptions(
        ranker=vectorsearch_v1beta.Ranker(
            rrf=vectorsearch_v1beta.ReciprocalRankFusion(weights=[1.0, 1.0])
        )
    ),
)

batch_results = data_object_search_service_client.batch_search_data_objects(batch_search_request)

print(f"Hybrid search results for '{query_text}' (Semantic + Text with built-in RRF):")
print("="*70)

# When a ranker is used, batch_results.results contains a single ranked list
# results[0] is the SearchDataObjectsResponse with the combined RRF-ranked results
if batch_results.results:
    # Get the first (and only) result which contains the RRF-ranked combined results
    combined_results = batch_results.results[0]
    
    for i, result in enumerate(combined_results.results[:10], 1):
        print(f"{i}. {result.data_object.data['name'][:55]}...")
        print(f"   {result.data_object.data['category']} | ${result.data_object.data['retail_price']:.2f}")
else:
    print("No results found")

print("\n💡 Hybrid search combines semantic understanding with keyword precision using built-in RRF!")
print("   Results are ranked by RRF score - products appearing high in both searches rank highest.")

The example above demonstrates practical hybrid search by combining semantic and text search with built-in RRF:

**How it works:**
1. **Same Query, Different Approaches**: Both searches use `"blue denim jeans"` but process it differently
   - **Semantic Search**: Understands meaning and context (e.g., "denim" and "jeans" are related)
   - **Text Search**: Looks for keyword matches in the product name field
2. **Built-in RRF Combining**: 
   - `BatchSearchDataObjectsRequest` executes both semantic and text searches in parallel.
   - The `combine` parameter with `ReciprocalRankFusion` automatically fuses the results.
   - The `weights` in `ReciprocalRankFusion` (here `[1.0, 1.0]`) determine the relative importance of each search in the fusion.

**RRF Algorithm Benefits:**
- Products ranking high in **both** searches get the highest combined scores
- Balances semantic understanding (meaning) with keyword precision (exact matches)
- Reduces impact of any single search strategy's weaknesses

-----

# Part 5: Production-Ready Search with ANN Indexes

In Part 4, we used **kNN (k-Nearest Neighbors)** search, which works instantly without any index creation. While kNN is perfect for development and small datasets, it has a critical limitation: **search latency increases significantly as your dataset grows**.

For production deployments with large-scale data (hundreds of thousands to billions of vectors), you need **ANN (Approximate Nearest Neighbor)** indexes.

## Why ANN Indexes?

**The Challenge with kNN:**
- kNN performs **brute-force comparison** against every vector in your Collection
- With 3,000 products: Fast (milliseconds)
- With 100,000 products: Slower (hundreds of milliseconds)
- With 1,000,000+ products: Too slow (seconds or more)

**The ANN Solution:**
ANN indexes use Google's [ScaNN (Scalable Nearest Neighbors)](https://github.com/google-research/google-research/tree/master/scann) algorithm - the same technology powering Google Search, YouTube, and Google Play - to enable **blazingly fast similarity search at billion-scale**.

## ANN vs kNN: The Trade-off

| Feature | kNN (Part 4) | ANN (Part 5) |
|---------|-------------|--------------|
| **Index Creation** | None (instant) | Required (5-60+ minutes) |
| **Search Latency** | Increases with data size | Sub-second even at billion scale |
| **Accuracy** | 100% exact | ~99% (approximate, configurable) |
| **Best For** | Development, small datasets | Production, large-scale deployments |
| **Dataset Size** | < tens of thousands | Hundreds of thousands to billions |

## Key Benefits of ANN Indexes

1. **Blazing Fast Search**: Sub-second latency even with billions of vectors
2. **Advanced Filtering**: Pre-filter by data fields (category, price, etc.) during vector search
3. **Optimized Storage**: Store frequently accessed fields directly in the index for faster retrieval
4. **Production-Ready**: Built on battle-tested Google technology powering major products

## What We'll Build

In this section, we'll create an **ANN index** on our product name embeddings (`name_dense_embedding`) to enable:
- Lightning-fast semantic product search at scale
- Filtered searches like "Find jeans under $100 similar to 'blue denim'"
- Production-ready performance for e-commerce applications

Let's get started!

## Creating an ANN Index for Dense Embeddings

Now let's create our first ANN index! This index will dramatically speed up semantic search on the `name_dense_embedding` field, making it ready for production-scale deployments.

In [ ]:
## Creating an ANN Index for Dense Embeddings
request = vectorsearch_v1beta.CreateIndexRequest(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}",
    index_id="name-dense-index",  # Use hyphens instead of underscores
    index={
        "index_field": "name_dense_embedding",  # Index the product name dense embeddings
        "filter_fields": ["category", "retail_price"],  # Enable filtering by category and price
        "store_fields": ["name"],  # Store product name for quick retrieval
    },
)
dense_index_lro = vector_search_service_client.create_index(request)
dense_index_operation_name = dense_index_lro.operation.name
print(f"✅ Creating dense ANN index on 'name_dense_embedding'")
print(f"   LRO: {dense_index_operation_name}")
print(f"   This operation takes several minutes. We'll poll it later.")
print(f"\n💡 Once ready, searches like 'Find products with similar name semantics' will be lightning fast!")
dense_index_operation_name

### Waiting for Index Creation

Index creation is an asynchronous operation (LRO = Long Running Operation). For small datasets, indexing takes 5-10 minutes. For larger datasets, it can take hours. Let's wait for the index to complete. In case when you face with the Colab timeout error (900 seconds), retry running this cell.

In [ ]:
print(f"Waiting for dense index LRO: {dense_index_lro.operation.name}")
dense_index_lro.result()
print("Dense index ready.")

### Inspecting the Created Index

Before we start searching, let's verify that our index was created successfully and check its configuration.

In [ ]:
# Get and inspect the created index
get_index_request = vectorsearch_v1beta.GetIndexRequest(
    name=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}/indexes/name-dense-index"
)
index_info = vector_search_service_client.get_index(get_index_request)

print("Index Information:")
print("="*70)
print(f"Name: {index_info.name}")
print(f"Index Field: {index_info.index_field}")
print(f"Filter Fields: {list(index_info.filter_fields) if index_info.filter_fields else 'None'}")
print(f"Store Fields: {list(index_info.store_fields) if index_info.store_fields else 'None'}")
print("\nThe index is ready to accelerate searches on the 'name_dense_embedding' field!")

## Searching with ANN Indexes

Now that our ANN index is ready, let's see how it works!

### Important: How Indexes Work

**Key Concept:** The index is a performance optimization layer that is automatically used when you search on an indexed field.

**Search Process:**
1. You create an ANN index on a specific embedding field (e.g., `name_dense_embedding`)
2. You search on the Collection using `semantic_search` or `vector_search`
3. If an index exists for the search field, it's automatically used for blazing-fast performance
4. If no index exists, kNN (brute-force) search is used instead

**The Difference:**
- **Without Index (kNN)**: Collection search uses brute-force comparison against all vectors
- **With Index (ANN)**: Collection search automatically uses the index for sub-second performance at billion scale

Think of it like a database index: you don't query the index directly, but it makes your queries faster.

In [ ]:
query_text = "blue denim jeans"

# Run semantic search on the Collection - the ANN index is automatically used
semantic_search_request = vectorsearch_v1beta.SearchDataObjectsRequest(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}",  # Search on Collection
    semantic_search=vectorsearch_v1beta.SemanticSearch(
        search_text=query_text,
        search_field="name_dense_embedding",  # The indexed field
        task_type="RETRIEVAL_QUERY",
        top_k=5,
        output_fields=vectorsearch_v1beta.OutputFields(data_fields=["name", "category", "retail_price"]),
    ),
)

results = data_object_search_service_client.search_data_objects(semantic_search_request)

print(f"ANN-Accelerated Search Results for '{query_text}':")
print("="*70)
for i, result in enumerate(results, 1):
    print(f"{i}. {result.data_object.data['name'][:60]}...")
    print(f"   {result.data_object.data['category']} - ${result.data_object.data['retail_price']:.2f}")

print("\nEvidence that ANN index is used:")
print(f"- Index 'name-dense-index' exists in the Collection")
print(f"- Index is configured for field: {index_info.index_field}")
print(f"- Search uses the same field: name_dense_embedding")
print("- When field names match, the index is automatically used for faster search")

### Key Takeaways: ANN Indexes

**What We Accomplished:**

1. **Created ANN Index**: Built a ScaNN-powered index on `name_dense_embedding`
2. **Automatic Acceleration**: Searches on the Collection now automatically use the index for better performance
3. **Same API**: The search API remains identical - you still use `semantic_search` on the Collection
4. **Transparent Upgrade**: No code changes needed from Part 4 to Part 5 - just faster performance

**How ANN Indexes Work:**

- **Not a separate search endpoint**: You search on the Collection, not on the index
- **Automatic optimization**: When searching an indexed field, the index is used automatically
- **Transparent upgrade**: No code changes needed - just faster performance
- **Production-ready**: Built on Google's ScaNN algorithm, the same technology powering Google Search

**Performance Benefits:**

- **Scalability**: Same code works for 3K products or 3 billion products
- **Speed**: Sub-second latency even at massive scale vs. seconds/minutes with kNN
- **Efficiency**: ScaNN algorithm provides approximate results with ~99% accuracy at much faster speeds

**When to Use ANN vs kNN:**

| Scenario | Use This | Why |
|----------|----------|-----|
| Development & prototyping | kNN (Part 4) | Instant - no index build wait time |
| Small datasets (< 10K rows) | kNN (Part 4) | Fast enough without indexing overhead |
| Production with large data | ANN (Part 5) | Worth the index build time for better performance |
| Billions of vectors | ANN (Part 5) | Only viable option for acceptable query latency |

**Note on Filtering:**
While we configured `filter_fields` during index creation, filtering with `semantic_search` is not currently supported in the API. Filtering is available when using `vector_search` with manually generated embeddings.

-----

# Part 6: Clean Up

## Important: Delete Resources to Avoid Costs

Vector Search 2.0 resources incur costs when active. To avoid unexpected charges, you must delete:

1. **ANN Indexes** (if created)
2. **Collections** (and all associated Data Objects)

**Cost Warning**: Leaving these resources running can result in significant charges. Always clean up when you're done with the tutorial!

## Cleanup Process

The cleanup must follow this order:
1. Delete all ANN indexes first
2. Then delete the Collection (this also deletes all Data Objects)

Let's clean up all resources created in this tutorial.

## Step 1: Delete ANN Index

First, we need to delete the ANN index. This is a long-running operation (LRO) that may take a few minutes.

In [ ]:
# Delete the ANN index
request = vectorsearch_v1beta.DeleteIndexRequest(
    name=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}/indexes/name-dense-index"
)
delete_index_lro = vector_search_service_client.delete_index(request)
print(f"🗑️ Deleting ANN index 'name-dense-index'...")
print(f"   LRO: {delete_index_lro.operation.name}")

In [ ]:
# Wait for index deletion to complete
print(f"Waiting for index deletion LRO: {delete_index_lro.operation.name}")
delete_index_lro.result()
print("✅ ANN index deleted successfully!")

## Step 2: Delete Collection

Now that the index is deleted, we can delete the Collection. This will also delete all Data Objects stored in the Collection.

In [ ]:
# Delete the Collection (and all Data Objects inside it)
request = vectorsearch_v1beta.DeleteCollectionRequest(
    name=f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/{collection_id}"
)
vector_search_service_client.delete_collection(request)
print(f"🗑️ Deleted Collection: {collection_id}")
print(f"   All Data Objects in the Collection have been deleted as well.")
print("\n✅ Cleanup complete! All Vector Search 2.0 resources have been deleted.")

## Cleanup Summary

You've successfully deleted all resources created in this tutorial:

✅ **ANN Index** (`name_dense_index`) - Deleted  
✅ **Collection** (`{collection_id}`) - Deleted  
✅ **All Data Objects** (3,000 products) - Deleted

-----

# Summary

Congratulations! You've completed a comprehensive introduction to **Vertex AI Vector Search 2.0**. Let's recap what you've learned and where to go next.

## What You've Learned

### 1. **Collections** - Schema-Enforced Data Containers
- Created Collections with data schemas (data) and vector schemas (embeddings)
- Configured auto-embeddings using Vertex AI's `text-embedding-004` model
- Learned how Collections enforce data structure for consistency

### 2. **Data Objects** - Your Actual Data
- Created individual Data Objects with data and vectors
- Used batch operations for efficient bulk imports (100x faster than individual creates)
- Implemented rate limiting to stay within API quotas (3,000 RPM for embeddings)
- Understood how auto-embeddings work with empty `vectors: {}`

### 3. **Querying and Filtering** - SQL-Like Metadata Search
- Used query operators (`$eq`, `$lt`, `$gte`, `$and`, `$or`) to filter data
- Retrieved products by category, price range, and complex conditions
- Performed aggregate queries to get collection statistics

### 4. **Vector Search** - Semantic Similarity with kNN
- **Semantic Search**: Natural language queries with auto-generated embeddings
- **Text Search**: Traditional keyword matching
- **Hybrid Search**: Combined semantic + text search with manual RRF ranking
- Learned when to use kNN (development, small datasets) vs when to scale to ANN

### 5. **Production-Ready Search** - ANN Indexes for Scale
- Created ANN indexes powered by Google's ScaNN algorithm
- Understood the kNN vs ANN trade-off (instant vs fast, exact vs approximate)
- Learned how indexes automatically accelerate searches on indexed fields
- Configured filter fields and store fields for optimized queries

### 6. **Resource Management** - Cost Control
- Properly deleted indexes before collections (order matters!)
- Understood that collection deletion cascades to all data objects
- Learned the importance of cleanup to avoid unexpected costs

## Key Takeaways

✅ **Unified Storage**: Vectors + user provided data together (no separate database needed)  
✅ **Auto-Embeddings**: Automatic generation via Vertex AI models  
✅ **Flexible Search**: Semantic, text, and hybrid search in one platform  
✅ **Development to Production**: Start with kNN, scale with ANN indexes  
✅ **Battle-Tested Technology**: Built on ScaNN (powers Google Search, YouTube, Google Play)  
✅ **Enterprise-Ready**: Scalability from 3K to 3B vectors with same API

## Architecture Recap

```
Collection (Schema-Enforced Container)
├── Data Schema (Data structure)
├── Vector Schema (Embedding configurations)
├── Data Objects (Individual items with user provided data + vectors)
└── Indexes (Optional performance optimization)
    ├── kNN (Instant, perfect for dev/small datasets)
    └── ANN (Production, billion-scale with sub-second latency)
```

## Get Started with Your Own Projects

### 1. **Documentation and Guides**
- [Vector Search 2.0 Overview](https://cloud.google.com/vertex-ai/docs/vector-search-2/overview)
- [Collections Guide](https://cloud.google.com/vertex-ai/docs/vector-search-2/collections/collections)
- [Data Objects Guide](https://cloud.google.com/vertex-ai/docs/vector-search-2/data-objects/data-objects)
- [Search Guide](https://cloud.google.com/vertex-ai/docs/vector-search-2/query-search/search)
- [Indexes Guide](https://cloud.google.com/vertex-ai/docs/vector-search-2/indexes/indexes)

### 2. **Additional Tutorials**
- [Vector Search 2.0 Quickstart](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/vector-search-2-quickstart.ipynb) - Includes sparse embeddings and true hybrid search with built-in RRF
- [Introduction to Text Embeddings and Vector Search](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro-textemb-vectorsearch.ipynb) - Learn the fundamentals of embeddings

### 3. **SDK and API References**
- [Python SDK Documentation](https://cloud.google.com/python/docs/reference/vectorsearch/latest)
- Install: `pip install google-cloud-vectorsearch`

### 4. **Sample Datasets**
- [TheLook E-Commerce Dataset](https://console.cloud.google.com/marketplace/product/bigquery-public-data/thelook-ecommerce) (used in this tutorial)
- Bring your own data: Any JSON-compatible data + text for embeddings

### 5. **Use Cases to Explore**
- **E-Commerce**: Product recommendations, semantic search, visual similarity
- **Content Discovery**: Article/video recommendations, duplicate detection
- **Customer Support**: FAQ matching, ticket routing, knowledge base search
- **Enterprise Search**: Document retrieval, code search, internal knowledge graphs

### 6. **Community and Support**
- [Google Cloud Community](https://www.googlecloudcommunity.com/)
- [Stack Overflow - google-cloud-platform](https://stackoverflow.com/questions/tagged/google-cloud-platform)
- [GitHub Issues](https://github.com/googleapis/python-vector-search/issues)

## Next Steps

1. **Experiment with the full dataset**: Change `MAX_PRODUCTS = None` in the data loading section to use all 30K products
2. **Try your own data**: Replace the TheLook dataset with your own JSON data
3. **Explore sparse embeddings**: Add sparse vector fields for true hybrid search with built-in RRF
4. **Build a production app**: Integrate Vector Search 2.0 into your application using the Python SDK
5. **Optimize for scale**: Create ANN indexes for production workloads with billions of vectors

---

**Ready to build something amazing?** Start by modifying this notebook with your own data, or explore the [Vector Search 2.0 documentation](https://cloud.google.com/vertex-ai/docs/vector-search-2/overview) for advanced features!

**Questions or feedback?** Open an issue on [GitHub](https://github.com/GoogleCloudPlatform/generative-ai/issues) or ask on [Stack Overflow](https://stackoverflow.com/questions/tagged/google-cloud-platform).